In [45]:
from json import dumps, loads
import yaml
from pathlib import Path
from fhir.resources import construct_fhir_element
from fhir.resources.observation import Observation as Obs
from fhir.resources.questionnaireresponse import QuestionnaireResponse as QR
from survey_obs import survey_obs

In [46]:
survey_name ='AHC-HRSN' #'PRAPARE' #'HVS' # 
survey_code = '96777-8' #'88121-9'  # don't need if already grouped like PRAPARE
survey_code_name = 'Accountable health communities (AHC) health-related social needs screening (HRSN) tool' #Hunger Vital Sign [HVS]' # don't need if already grouped like PRAPARE
create_survey_panel = True  #|False if already grouped in QR like PRAPARE
survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-AHC-HRSN-example.yml'
#survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-PRAPARE-example.yml'
#survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-hunger-vital-sign-example.yml'
#out_dir = Path.cwd() / 'output'
out_dir = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/'
path = Path(survey_path)

out_dir

'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/'

### Fetch Panels Observations and add hasMembers

In [47]:
def update_members(parent, members=[]):
    try:
        parent = parent.replace("/", "")
    except AttributeError:
        pass
    in_path = Path().cwd() / 'output' / f'Observation-{survey_name}-panel-example-{parent}.yml'
    my_obj = Obs.parse_file(in_path)
    # print(my_obj.yaml(indent=True))
    my_obj.hasMember = []
    for m in members:
        new_member = dict(reference = f'Observation/{m}')
        my_obj.hasMember.append(new_member)
    # print('='*80)
    # print(my_obj.yaml(indent=True))
    # print('='*80)
    write_out(my_obj)

### Write Observations to file

In [48]:
def write_out(fhir_obj):
    out_path = Path(out_dir) / f'Observation-{fhir_obj.id}.yml'
    # note need to the date time format is not json serializabel out of the box see QR-OBs-DE/YAML-JSON-date-serialization.ipynb
    out_path.write_text(fhir_obj.yaml())

### Fetch QuestionnaireResponse Instance from file

In [49]:
from fhir.resources.questionnaireresponse import QuestionnaireResponse as QR
qr_obj = QR.parse_file(path)
qr_obj.authored.isoformat()

'2022-03-29T22:38:59.084000+00:00'

### Create Observation Instance with the metadata

In [50]:
def get_coding(my_code):
    coding_obj = construct_fhir_element('Coding',my_code.valueCoding)
    return coding_obj

def get_ccode(my_coding):
    ccode_obj = construct_fhir_element('CodeableConcept',{})
    
    ccode_obj.coding = [my_coding]
    ccode_obj.text = my_coding.display

    return(ccode_obj)

def get_answers(obs_obj, answer):
    if answer.valueCoding:
      # print(answer.valueCoding)  
      obs_obj.valueCodeableConcept = get_ccode(answer.valueCoding)
    elif answer.valueString:
      obs_obj.valueString = answer.valueString 
    elif answer.valueDecimal:
      obs_obj.valueString  = answer.valueDecimal 
    #print(obs_obj.json(indent=True))
    return (obs_obj)


def create_obs (obs_id, text, obs_type='item', answers = None):
    obs_obj = Obs.parse_file("survey_obs.yml")
    obs_obj.id = f'{survey_name}-{obs_type}-example-{obs_id}'
    obs_obj.meta.extension[0].valueString = f'{survey_name} {obs_type} Example {obs_id}'.title()
    obs_obj.meta.extension[1].valueMarkdown = f'This is a {obs_obj.meta.extension[0].valueString} ({text}) \
for the *US Core Screening Response Observation Profile*'
    # obs_obj.code.coding[0].system = 'http://loinc.org'
    obs_obj.code.coding[0].code = obs_id
    obs_obj.code.coding[0].display = text
    obs_obj.code.text = text
    if answers:
        get_answers(obs_obj, answers)
        obs_obj.hasMember = None
    obs_obj.effectiveDateTime = qr_obj.authored.isoformat()
    obs_obj.derivedFrom[0].reference = f'QuestionnaireResponse/{qr_obj.id}'
    obs_obj.derivedFrom[0].display = qr_obj.meta.extension[0].valueString
    return(obs_obj)

### Use recursion to parse out the nested responses


- Create Observations only from those with values

In [51]:
def get_items(my_item, parent = None):

    try:
      assert my_item.item
    except (AttributeError, AssertionError):
        return  # item observations !!

    else: # groups observations !!

      my_list = []
      for i in my_item.item:  # <<< create new Observation here
        new_item = construct_fhir_element('QuestionnaireResponseItem', i)
        print(new_item.linkId)
        print(new_item.text)
        # get_answers(new_item.answer)
        try:
          assert new_item.answer
        except AssertionError:
            print ('NO Answer?  - group?\n\n')
            survey_obs_type = "panel"
            new_code = new_item.linkId.split("/")[-1]
            my_panel_obs = create_obs(new_code, new_item.text, survey_obs_type)
            my_list.append(my_panel_obs.id)
            write_out(my_panel_obs)

        else:
          print ('Answer  - item!\n\n')
          if len(new_item.answer) > 1:
            survey_obs_type = "multiselect-item"
            for i, answer in enumerate(new_item.answer):
                new_code = f'{new_item.linkId.split("/")[-1]}-answer{i}'
                my_item_obs = create_obs(new_code, new_item.text, survey_obs_type, answer)
                my_list.append(my_item_obs.id)
                write_out(my_item_obs)
          else:
            survey_obs_type = "item"
            new_code = new_item.linkId.split("/")[-1]
            my_item_obs = create_obs(new_code, new_item.text, survey_obs_type, new_item.answer[0])
            my_list.append(my_item_obs.id)
            write_out(my_item_obs)
          
        get_items(i, new_item.linkId)
        
        
      print(f'my_list for {parent} = {my_list}\n\n')
      try:
        update_members(parent, my_list)
      except Exception as e:
        print(e)
        if create_survey_panel:
            my_panel_obs = create_obs(survey_code, survey_code_name, "panel")
            write_out(my_panel_obs)
            update_members(survey_code, my_list)

get_items(qr_obj)




/76513-1
How hard is it for you to pay for the very basics like food, housing, medical care, and heating?
Answer  - item!


/96780-2
Do you want help finding or keeping work or a job?
Answer  - item!


/96781-0
If for any reason you need help with day-to-day activities such as bathing, preparing meals, shopping, managing finances, etc., do you get the help you need?
Answer  - item!


/93159-2
How often do you feel lonely or isolated from those around you?
Answer  - item!


/97027-7
Do you speak a language other than English at home?
Answer  - item!


/96782-8
Do you want help with school or training? For example, starting or completing job training or getting a high school diploma, GED or equivalent.
Answer  - item!


/89555-7
In the last 30 days, other than the activities you did for work, on average, how many days per week did you engage in moderate exercise (like walking fast, running, jogging, dancing, swimming, biking, or other similar activities)?
Answer  - item!


/68516-4
On av